In [1]:
# importar pacotes necessários
import numpy as np
import pandas as pd

In [2]:
# definir parâmetros extras
pd.set_option('precision', 4)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
# importar pacotes usados na seleção do modelo e na medição da precisão
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

# importar os pacotes necessários para os algoritmos de regressão
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lars
from sklearn.linear_model import LassoLars
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import ARDRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import TheilSenRegressor
from sklearn.linear_model import HuberRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.svm import NuSVR
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor

from xgboost import XGBRegressor

In [4]:
input_prefix = ''
#input_prefix = 'https://github.com/hjort/ai-labs/raw/master/jupyter/sales-russia/'

bzfile = '.bz2' # ''

## Training

In [5]:
df_train = pd.read_csv(input_prefix + 'train-data.csv' + bzfile, nrows=500000,
                       index_col='id')
print('shape:', df_train.shape)
df_train.head()

shape: (500000, 11)


,shop_id,item_id,date_block_num,item_cnt_month,city_code,item_category_id,subject_code,group_code,subgroup_code,year,month
id,,,,,,,,,,,
0,5,5037,0,0.0,5,19,793,6,13,2013,1
1,5,5037,1,0.0,5,19,793,6,13,2013,2
2,5,5037,2,0.0,5,19,793,6,13,2013,3
3,5,5037,3,0.0,5,19,793,6,13,2013,4
4,5,5037,4,0.0,5,19,793,6,13,2013,5


In [6]:
#for col in ('shop_id', 'item_id'):
#    df_train[col] = df_train[col].astype('category')

In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500000 entries, 0 to 499999
Data columns (total 11 columns):
shop_id             500000 non-null int64
item_id             500000 non-null int64
date_block_num      500000 non-null int64
item_cnt_month      500000 non-null float64
city_code           500000 non-null int64
item_category_id    500000 non-null int64
subject_code        500000 non-null int64
group_code          500000 non-null int64
subgroup_code       500000 non-null int64
year                500000 non-null int64
month               500000 non-null int64
dtypes: float64(1), int64(10)
memory usage: 45.8 MB


In [8]:
df_train.describe()

,shop_id,item_id,date_block_num,item_cnt_month,city_code,item_category_id,subject_code,group_code,subgroup_code,year,month
count,500000.0000,500000.0000,500000.0000,500000.0000,500000.0000,500000.0000,500000.0000,500000.0000,500000.0000,500000.0000,500000.0000
mean,4.9596,11078.0863,16.4999,0.1961,4.9596,46.2516,2123.7684,11.2685,23.0962,2013.9412,6.2059
std,0.8072,6207.9194,9.8107,2.0144,0.8072,16.6747,1383.5725,2.8357,17.5663,0.8022,3.3235
min,4.0000,30.0000,0.0000,-1.0000,4.0000,0.0000,1.0000,1.0000,1.0000,2013.0000,1.0000
25%,4.0000,5621.0000,8.0000,0.0000,4.0000,37.0000,823.0000,11.0000,8.0000,2013.0000,3.0000
50%,5.0000,11265.0000,16.0000,0.0000,5.0000,44.0000,2147.0000,12.0000,19.0000,2014.0000,6.0000
75%,6.0000,16075.0000,25.0000,0.0000,6.0000,58.0000,3099.0000,13.0000,39.0000,2015.0000,9.0000
max,6.0000,22167.0000,33.0000,294.0000,6.0000,83.0000,4655.0000,18.0000,55.0000,2015.0000,12.0000


## Testing

In [9]:
df_test = pd.read_csv(
    input_prefix + 'test-data.csv' + bzfile, nrows=100000,
    index_col='id')
print('shape:', df_test.shape)
df_test.head()

shape: (100000, 10)


,shop_id,item_id,date_block_num,city_code,item_category_id,subject_code,group_code,subgroup_code,year,month
id,,,,,,,,,,
0,5,5037,34,5,19,793,6,13,2015,11
1,5,5320,34,5,55,818,13,6,2015,11
2,5,5233,34,5,19,786,6,13,2015,11
3,5,5232,34,5,23,786,6,19,2015,11
4,5,5268,34,5,20,786,6,14,2015,11


In [10]:
#for col in ['shop_id', 'item_id']:
#    df_test[col] = df_test[col].astype('category')

In [11]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 10 columns):
shop_id             100000 non-null int64
item_id             100000 non-null int64
date_block_num      100000 non-null int64
city_code           100000 non-null int64
item_category_id    100000 non-null int64
subject_code        100000 non-null int64
group_code          100000 non-null int64
subgroup_code       100000 non-null int64
year                100000 non-null int64
month               100000 non-null int64
dtypes: int64(10)
memory usage: 8.4 MB


In [12]:
df_test.describe()

,shop_id,item_id,date_block_num,city_code,item_category_id,subject_code,group_code,subgroup_code,year,month
count,100000.0000,100000.0000,100000.0,100000.0000,100000.0000,100000.0000,100000.0000,100000.0000,100000.0,100000.0
mean,15.3280,11007.8515,34.0,10.3280,46.2732,2110.2089,11.2715,23.2928,2015.0,11.0
std,8.8727,6255.5231,0.0,4.5595,16.7180,1391.8800,2.8321,17.6705,0.0,0.0
min,2.0000,30.0000,34.0,2.0000,0.0000,1.0000,1.0000,1.0000,2015.0,11.0
25%,6.0000,5365.0000,34.0,6.0000,37.0000,793.0000,11.0000,8.0000,2015.0,11.0
50%,15.0000,11181.0000,34.0,11.0000,43.0000,2114.0000,12.0000,19.0000,2015.0,11.0
75%,24.0000,16069.0000,34.0,15.0000,58.0000,3099.0000,13.0000,39.0000,2015.0,11.0
max,31.0000,22167.0000,34.0,15.0000,83.0000,4655.0000,18.0000,55.0000,2015.0,11.0


In [ ]:
data = df_train

In [ ]:
# definir dados de entrada
X = data.drop(['item_cnt_month'], axis=1) # tudo, exceto a coluna alvo
y = data['item_cnt_month'] # apenas a coluna alvo

print('Forma dos dados originais:', X.shape, y.shape)

## Treinamento dos modelos preditivos

In [ ]:
MSE = 'neg_mean_squared_error'

models = {}

parallel_jobs = 8 # paralelização

In [ ]:
from datetime import datetime

# avalia o desempenho do modelo, retornando o valor do MSE
def evaluate_model_cv(name, model, X=X, y=y):
    start = datetime.now()
    kfold = KFold(n_splits=10, random_state=42)
    results = cross_val_score(model, X, y, cv=kfold, scoring=MSE, verbose=1, n_jobs=parallel_jobs)
    end = datetime.now()
    elapsed = int((end - start).total_seconds() * 1000)
    score = (-1) * results.mean()
    stddev = results.std()
    models[name] = {'model': model, 'score': score, 'stddev': stddev, 'elapsed': elapsed}
    print(model, '\nScore: %.2f (+/- %.2f) [%5s ms]' % (score, stddev, elapsed))
    return score, stddev, elapsed

In [ ]:
# faz o ajuste fino do modelo, calculando os melhores hiperparâmetros
def fine_tune_model(model, params, X=X, y=y):
  print('\nFine Tuning Model:')
  print(model, "\nparams:", params)
  kfold = KFold(n_splits=10, random_state=42)
  grid = GridSearchCV(estimator=model, param_grid=params, scoring=MSE, cv=kfold, verbose=1, n_jobs=parallel_jobs)
  grid.fit(X, y)
  print('\nGrid Best Score: %.2f' % (grid.best_score_ * (-1)))
  print('Best Params:', grid.best_params_)
  return grid

### Avaliação e ajuste fino de cada modelo preditivo

- https://scikit-learn.org/stable/modules/classes.html

In [ ]:
model = KNeighborsRegressor(n_jobs=-1, n_neighbors=5, weights='distance')
%time evaluate_model_cv('KNN', model)

#n_neighbors=5, weights=’uniform’, algorithm=’auto’, leaf_size=30, p=2, metric=’minkowski’,
#metric_params=None, n_jobs=None

params = {
    'n_neighbors': [1, 3, 5, 7, 9, 11, 13],
    'weights': ['uniform', 'distance']
}
#fine_tune_model(model, params)

In [ ]:
model = RandomForestRegressor(random_state=42, n_jobs=-1, n_estimators=50, max_depth=9)
%time evaluate_model_cv('RF', model)

#n_estimators=’warn’, criterion=’mse’, max_depth=None, min_samples_split=2, min_samples_leaf=1, 
#min_weight_fraction_leaf=0.0, max_features=’auto’, max_leaf_nodes=None, min_impurity_decrease=0.0, 
#min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, 
#verbose=0, warm_start=False

params = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None, 5, 9, 13]
}
#fine_tune_model(model, params)

In [ ]:
model = GradientBoostingRegressor(random_state=42,
    learning_rate=0.05, n_estimators=100, max_depth=4, max_features=0.85)
#%time evaluate_model_cv('GB', model)

#loss=’ls’, learning_rate=0.1, n_estimators=100, subsample=1.0, criterion=’friedman_mse’, min_samples_split=2,
#min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, 
#min_impurity_split=None, init=None, random_state=None, max_features=None, alpha=0.9, verbose=0, 
#max_leaf_nodes=None, warm_start=False, presort=’auto’, validation_fraction=0.1, n_iter_no_change=None, 
#tol=0.0001

params = dict(
    n_estimators=[100, 250],
    max_features=[0.75, 0.85, 1.0],
    max_depth=[4, 8],
    learning_rate=[0.05, 0.1, 0.15],
    #subsample=[0.4, 0.6, 0.8]
)
fine_tune_model(model, params)

In [ ]:
model = ExtraTreesRegressor(random_state=42, n_jobs=-1, n_estimators=75, max_features=0.85)
#%time evaluate_model_cv('ET', model)

#n_estimators=’warn’, criterion=’mse’, max_depth=None, min_samples_split=2, min_samples_leaf=1, 
#min_weight_fraction_leaf=0.0, max_features=’auto’, max_leaf_nodes=None, min_impurity_decrease=0.0, 
#min_impurity_split=None, bootstrap=False, oob_score=False, n_jobs=None, random_state=None, verbose=0,
#warm_start=False

params = dict(
    n_estimators=[50, 75, 100, 200],
    max_features=['auto', 0.75, 0.85, 1.0]
)
fine_tune_model(model, params)

In [ ]:
model = BaggingRegressor(random_state=42, n_jobs=-1, base_estimator=DecisionTreeRegressor(), max_features=0.5, n_estimators=200)
#%time evaluate_model_cv('BG', model)

#base_estimator=None, n_estimators=10, max_samples=1.0, max_features=1.0, bootstrap=True, 
#bootstrap_features=False, oob_score=False, warm_start=False, n_jobs=None, random_state=None, verbose=0

params = dict(
    n_estimators=[50, 75, 100, 200],
    max_features=[0.5, 0.75, 1.0]
)
fine_tune_model(model, params)

In [ ]:
model = AdaBoostRegressor(random_state=42, n_estimators=100, base_estimator=DecisionTreeRegressor())
#%time evaluate_model_cv('ABDT', model)

# base_estimator=None, n_estimators=50, learning_rate=1.0, algorithm=’SAMME.R’, random_state=None

params = dict(
    n_estimators=[50, 75, 100, 200]
)
fine_tune_model(model, params)

## Comparação final entre os algoritmos

In [ ]:
results = []
names = []
scores = []
stddevs = []
times = []

best_model = None
highest_score = None

for name, model in models.items():
    score, stddev, elapsed = model['score'], model['stddev'], model['elapsed']
    results.append((score, stddev))
    names.append(name)
    scores.append(score)
    stddevs.append(stddev)
    times.append(elapsed)
    
    if not highest_score or score < highest_score:
        best_model = model['model']
        highest_score = score

In [ ]:
results_df = pd.DataFrame({'Model Name': names, 'Score': scores, 'Std Dev': stddevs, 'Time (ms)': times})
results_df.sort_values(by=['Score', 'Time (ms)'], ascending=[True, False])

In [ ]:
print('Best Model:\n\n%s' % (best_model))

In [ ]:
model = best_model
model.fit(X, y)

In [ ]:
X.head()

In [ ]:
del(X)
del(y)

## ....

In [ ]:
# colocar as colunas na mesma ordem dos dados de treino
cols = df_train.columns.drop(['item_cnt_month'])
cols

In [ ]:
# selecionar atributos para o modelo
X_pred = df_test[cols]

In [ ]:
# realizar a previsão
y_pred = best_model.predict(X_pred)

In [ ]:
df_final = pd.DataFrame(data=y_pred, columns=['item_cnt_month'], index=df_test.index)
df_final.index.names = ['ID']
df_final.head()

In [ ]:
df_final.info()

In [ ]:
!test -d submissions/ || mkdir submissions/

In [ ]:
df_final.to_csv('submissions/submission.csv')